In [ ]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [ ]:
df = pd.read_csv("data/immo_data_202208_v2_imputed.csv")

In [ ]:
# Clean and prepare data
df['Living space'] = df['Living space'].str.replace(' m²', '').astype(float)

In [ ]:
df.head()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.info()

In [ ]:
df = df[~df["price_cleaned"].isna()]

In [ ]:
df= df.drop(columns=["price", "Living space"])

df= df.rename(columns={"price_cleaned": "price", "Living_space_merged": "Living_space"})


# LE 1
Performance testen mit Häuserpreise pro gemeinde oder alle Datenpunkte direkt laden.

In [ ]:
import numpy as np
import pandas as pd

multiplier = 10
n = df.shape[0]
augmented_list = [df]
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Schweizer Koordinaten-Grenzen
lat_min, lat_max = 45.8, 47.9
lon_min, lon_max = 5.9, 10.5

for i in range(multiplier - 1):
    df_syn = df.sample(n=n, replace=True, random_state=42 + i).reset_index(drop=True)

    for col in numeric_cols:
        if col in ['lat', 'lon']:
            noise = np.random.randn(n) * 0.01  # kleine Variation
            if col == 'lat':
                df_syn[col] = (df_syn[col] + noise).clip(lat_min, lat_max)
            else:
                df_syn[col] = (df_syn[col] + noise).clip(lon_min, lon_max)
        else:
            factor = 1 + np.random.randn(n) * 0.01
            df_syn[col] = (df_syn[col] * factor).clip(lower=0)

    augmented_list.append(df_syn)

df_augmented = pd.concat(augmented_list, ignore_index=True)


In [ ]:
df_augmented.info()

In [ ]:
import time
from dash import html, dcc, dash_table, Input, Output, State, callback
import dash_bootstrap_components as dbc
import matplotlib.pyplot as plt

# Define fractions of data to test (e.g., 10%, 50%, 100%)
fractions = [0.1,0.25, 0.5,0.75, 1.0]
repeats = 10
perf_times = {frac: [] for frac in fractions}
map_figs = {}

# Measure startup time for each fraction, repeated trials
for frac in fractions:
    for i in range(repeats):
        t0 = time.perf_counter()
        df_sub = df_augmented.sample(frac=frac, random_state=42)
        fig_map = px.scatter_mapbox(
            df_sub,
            lat='lat', lon='lon',
            color='price', size='Living_area_unified', size_max=15,
            hover_name='Municipality', hover_data=['type_unified', 'price'],
            mapbox_style='carto-positron', zoom=6
        )
        elapsed = time.perf_counter() - t0
        perf_times[frac].append(elapsed)
    # store the full-data map from the first repeat
    if frac == 1.0:
        map_figs['full'] = fig_map

# Compute means and standard deviations
means = [sum(perf_times[frac]) / repeats for frac in fractions]
stds = [(pd.Series(perf_times[frac]).std()) for frac in fractions]
percentages = [int(f * 100) for f in fractions]

# Plot performance with error bars using Matplotlib
plt.figure(figsize=(8, 5))
plt.errorbar(percentages, means, yerr=stds, marker='o', linestyle='-')
plt.title('Zeitdauer der Karte vs. Datenmenge')
plt.xlabel('Datenmenge (%)')
plt.ylabel('Zeitdauer (Sekunden)')
plt.grid(True)
plt.tight_layout()
plt.show()

# Initialize Dash app for map visualization only
app = dash.Dash(__name__)

# Build layout with just the full-data map
ing_app = html.Div([
    html.H1('Swiss Real Estate Map Performance'),
    dcc.Graph(figure=map_figs['full'])
], style={'width': '90%', 'margin': '0 auto'})

app.layout = ing_app


In [ ]:
#app.run(debug=False, port=8049)

## Karte normal

In [ ]:
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# Use a Bootstrap theme
external_stylesheets = [dbc.themes.LUX]

# Karte vorbereiten (keine Filter/Slider mehr)
fig_map = px.scatter_mapbox(
    df_augmented,
    lat='lat',
    lon='lon',
    color='price',
    size='Living_area_unified',
    size_max=15,
    hover_name='Municipality',
    hover_data=['type_unified', 'price'],
    mapbox_style='carto-positron',
    zoom=6.5
)
fig_map.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server  # für Deployment

# Navbar
navbar = dbc.NavbarSimple(
    brand="🏠 Schweizer Immobilienpreis-Übersicht",
    color="primary",
    dark=True,
    fluid=True,
)


# Layout: nur Navbar, Karte, Footer
app.layout = dbc.Container(
    [
        navbar,
        # Vollbreite Karte ohne Controls
        dbc.Row(
            dbc.Col(dcc.Graph(id='map-overview', figure=fig_map), width=12),
            className="mb-4"
        )
    ],
    fluid=True,
    className="p-4"
)

app.run(debug=True, port=8041)


## LOD

In [ ]:
df_augmented.info()

In [ ]:
import time
import geopandas as gpd
import json
from dash import Dash, dcc, html
import plotly.express as px
from flask_caching import Cache

# --- 1) Dash-App und Cache konfigurieren ---
app = Dash(__name__)
server = app.server  # für Deployment
cache = Cache(app.server, config={
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': 'cache-directory',
    'CACHE_THRESHOLD': 1  # maximal ein gecachter Eintrag
})

# Pfad zur GPKG-Datei und Geometrietoleranz (Größe in Grad)
GPKG_PATH = "swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"
LAYER_NAME = "tlm_hoheitsgebiet"
SIMPLIFY_TOLERANCE = 0.001

# --- 2) Geodaten laden, verarbeiten & cachen ---
@cache.memoize(timeout=3600)
def load_and_prepare_geojson():
    # 2.1 Geo-Daten einlesen und auf WGS84 umprojizieren
    gdf = gpd.read_file(GPKG_PATH, layer=LAYER_NAME).to_crs(epsg=4326)

    # 2.2 Durchschnittspreis pro Gemeinde berechnen
    avg_price = (
        df_augmented
        .groupby("Municipality", as_index=False)["price"]
        .mean()
        .rename(columns={"price": "avg_price"})
    )

    # 2.3 Merge und geometrische Vereinfachung
    gdf_merged = gdf.merge(
        avg_price,
        left_on="name",
        right_on="Municipality",
        how="left"
    )
    gdf_simple = gdf_merged[["name", "avg_price", "geometry"]]
    gdf_simple["geometry"] = (
        gdf_simple.geometry
        .simplify(tolerance=SIMPLIFY_TOLERANCE, preserve_topology=True)
    )

    # 2.4 In GeoJSON konvertieren
    return json.loads(gdf_simple.to_json())

# Lade GeoJSON einmal aus dem Cache
geojson_data = load_and_prepare_geojson()

# --- 3) Choroplethen-Karte erstellen ---
fig_choro = px.choropleth_mapbox(
    locations=[f["properties"]["name"] for f in geojson_data["features"]],
    geojson=geojson_data,
    featureidkey="properties.name",
    color=[f["properties"]["avg_price"] for f in geojson_data["features"]],
    mapbox_style="open-street-map",
    zoom=6,
    center={"lat": 46.8, "lon": 8.2},
    opacity=0.6,
    color_continuous_scale="Viridis",
    labels={"color": "Durchschnittspreis (CHF)"}
)
fig_choro.update_layout(margin={"r":0, "t":0, "l":0, "b":0})

# --- 4) Layout definieren und Server starten ---
app.layout = html.Div([
    html.H1("Durchschnittspreis pro Gemeinde (CH)"),
    dcc.Graph(id="price-map", figure=fig_choro)
])

if __name__ == '__main__':
    app.run(debug=True, port=8040)


# LE 2

In [ ]:
import dash
from dash import html, dcc, dash_table, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np
import pandas as pd



# Bootstrap-Theme
external_stylesheets = [dbc.themes.LUX]

# Schrittweite für Slider
def get_price_bounds(df, step=250_000):
    raw_min = df['price'].min()
    raw_max = df['price'].max()
    slider_min = int((raw_min // step) * step)
    slider_max = int(np.ceil(raw_max / step) * step)
    return slider_min, slider_max

step = 10_000_000
slider_min, slider_max = get_price_bounds(df, step)

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

# Navbar
def create_navbar():
    return dbc.NavbarSimple(
        brand="🏠 Schweizer Immobilien-Dashboard",
        color="primary",
        dark=True,
        fluid=True,
    )

# KPI-Karten
def create_kpi_cards():
    cards = [
        dbc.Card(dbc.CardBody([html.H6("Anzahl Inserate"), html.H2(id="kpi-count")])),
        dbc.Card(dbc.CardBody([html.H6("Durchschnittspreis"), html.H2(id="kpi-avgprice")])),
        dbc.Card(dbc.CardBody([html.H6("Durchschnittliche Wohnfläche"), html.H2(id="kpi-avgarea")]))
    ]
    return dbc.Row([dbc.Col(card, md=4) for card in cards], className="mb-4")

# Filter-Controls
filter_card = dbc.Card([
    dbc.CardHeader(html.H5("Filter")),
    dbc.CardBody([
        dbc.Label("Gemeinde", html_for='municipality-dropdown'),
        dcc.Dropdown(
            id='municipality-dropdown',
            options=[{'label': m, 'value': m} for m in sorted(df['Municipality'].dropna().unique())],
            multi=True,
            placeholder='Auswählen…'
        ),
        html.Br(),
        dbc.Label("Immobilientyp", html_for='type-dropdown'),
        dcc.Dropdown(
            id='type-dropdown',
            options=[{'label': t, 'value': t} for t in sorted(df['type_unified'].dropna().unique())],
            multi=True,
            placeholder='Auswählen…'
        ),
        html.Br(),
        dbc.Label("Preisspanne (CHF)", html_for='price-slider'),
        html.Div(
            dcc.RangeSlider(
                  id='price-slider',
                  min=slider_min,
                  max=slider_max,
                  value=[slider_min, slider_max],
                  step=step,
                  tooltip={'placement': 'bottom'},
                  allowCross=False
                ),
            style={'width': '100%', 'padding': '0 10px'}
        )
    ])
], className="mb-4")

# Footer
def create_footer():
    return dbc.Container(
        html.Footer("2025 Einblicke in den Schweizer Immobilienmarkt", className="text-center text-muted py-3"),
        fluid=True
    )

# Layout
tmp_layout = [
    create_navbar(),
    create_kpi_cards(),
    dbc.Row([
        dbc.Col(filter_card, md=4),
        dbc.Col(dcc.Graph(id='map-overview', style={'height': '60vh'}), md=8)
    ], className="mb-4"),
    dbc.Row([
        dbc.Col(dcc.Graph(id='histogram-price'), md=6),
        dbc.Col(dcc.Graph(id='scatter-overview', config={'modeBarButtonsToAdd': ['lasso2d']}), md=6)
    ], className="mb-4"),
    dbc.Collapse(
        dash_table.DataTable(
            id='details-table',
            columns=[
                {'name': 'Gemeinde', 'id': 'Municipality'},
                {'name': 'Immobilientyp', 'id': 'type_unified'},
                {'name': 'Wohnfläche (m²)', 'id': 'Living_area_unified'},
                {'name': 'Preis (CHF)', 'id': 'price'},
                {'name': 'Beschreibung', 'id': 'description'},
                {'name': 'URL', 'id': 'url', 'presentation': 'markdown'}
            ],
            page_size=5,
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
            style_cell={'padding': '8px', 'textAlign': 'left'},
            row_selectable='single',
            cell_selectable=False
        ),
        id='details-collapse',
        is_open=True
    ),
    create_footer()
]

app.layout = dbc.Container(tmp_layout, fluid=True, className="p-4")

# Datenfilter-Funktion
def filter_df(municipalities, types, price_range):
    dff = df[df['price'].between(price_range[0], price_range[1])]
    if municipalities:
        dff = dff[dff['Municipality'].isin(municipalities)]
    if types:
        dff = dff[dff['type_unified'].isin(types)]
    return dff

# KPI-Callback
@app.callback(
    [Output('kpi-count','children'), Output('kpi-avgprice','children'), Output('kpi-avgarea','children')],
    [Input('municipality-dropdown','value'), Input('type-dropdown','value'), Input('price-slider','value')]
)
def update_kpis(munis, types, price_rng):
    dff = filter_df(munis, types, price_rng)
    count = len(dff)
    avg_price = f"CHF {dff['price'].mean():,.0f}" if count else "-"
    avg_area = f"{dff['Living_area_unified'].mean():.0f} m²" if count else "-"
    return count, avg_price, avg_area

# Graph-Update-Callback
@app.callback(
    [Output('map-overview','figure'), Output('histogram-price','figure'), Output('scatter-overview','figure')],
    [Input('municipality-dropdown','value'), Input('type-dropdown','value'), Input('price-slider','value')]
)
def update_graph(municipalities, types, price_range):
    dff = filter_df(municipalities, types, price_range)
    dff_plot = dff.rename(columns={'Municipality':'Gemeinde','type_unified':'Immobilientyp','price':'Preis (CHF)','Living_area_unified':'Wohnfläche (m²)','description':'Beschreibung'})
    fig_map = px.scatter_mapbox(
        dff_plot, lat='lat', lon='lon', color='Preis (CHF)', size='Wohnfläche (m²)', size_max=15,
        hover_name='Gemeinde', hover_data={'Immobilientyp':True,'Preis (CHF)':True,'lat':False,'lon':False},
        mapbox_style='carto-positron', zoom=6, labels={'Preis (CHF)':'Preis','Wohnfläche (m²)':'Fläche'}
    )
    fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
    fig_hist = px.histogram(dff_plot, x='Preis (CHF)', title='Preisverteilung (CHF)')
    fig_hist.update_layout(margin={'r':0,'t':30,'l':0,'b':0}, clickmode='event+select')
    fig_scatter = px.scatter(
        dff_plot, x='Wohnfläche (m²)', y='Preis (CHF)', color='Gemeinde', hover_name='Gemeinde', hover_data={'Wohnfläche (m²)':True,'Preis (CHF)':True}, labels={'Wohnfläche (m²)':'Fläche (m²)','Preis (CHF)':'Preis'}
    )
    fig_scatter.update_traces(marker=dict(opacity=0.7, line=dict(width=0.5)))
    fig_scatter.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, xaxis_title='Wohnfläche (m²)', yaxis_title='Preis (CHF)')
    return fig_map, fig_hist, fig_scatter

# Histogramm-Click → Slider
@app.callback(
    Output('price-slider','value'), [Input('histogram-price','clickData')], [State('price-slider','value')]
)
def hist_to_slider(clickData, current_value):
    if not clickData:
        return current_value
    start = int(clickData['points'][0]['x'])
    return [start, min(start+step, slider_max)]

# Klick-Auswahl → Tabelle anzeigen
@app.callback(
    [Output('details-table','data'), Output('details-collapse','is_open')],
    [Input('scatter-overview','clickData'), Input('scatter-overview','selectedData')],
    [State('details-collapse','is_open')]
)
def display_details(clickData, selectedData, is_open):
    # Klick auf einzelnen Punkt hat Vorrang
    if clickData and 'points' in clickData:
        pt = clickData['points'][0]
        x_val = pt['x']
        y_val = pt['y']
        sel = df[(df['Living_area_unified'] == x_val) & (df['price'] == y_val)]
        return sel[['Municipality','type_unified','Living_area_unified','price','description','url']].to_dict('records'), True
    # Lasso-/Box-Selection
    if selectedData and 'points' in selectedData:
        xs = [p['x'] for p in selectedData['points']]
        ys = [p['y'] for p in selectedData['points']]
        sel = df[df['Living_area_unified'].isin(xs) & df['price'].isin(ys)]
        return sel[['Municipality','type_unified','Living_area_unified','price','description','url']].to_dict('records'), True
    # Kein Klick/Selection
    return [], False

if __name__ == '__main__':
    app.run(debug=True, port=8052)


# LE 3

In [ ]:
import dash
from dash import html, dcc, dash_table, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np
import pandas as pd


external_stylesheets = [dbc.themes.LUX]
step = 10_000_000

# Grenzwerte für Slider
def get_price_bounds(df, step=step):
    raw_min = df['price'].min()
    raw_max = df['price'].max()
    slider_min = int((raw_min // step) * step)
    slider_max = int(np.ceil(raw_max / step) * step)
    return slider_min, slider_max

slider_min, slider_max = get_price_bounds(df)

# Layout-Komponenten
def create_navbar():
    return dbc.NavbarSimple(
        brand="🏠 Schweizer Immobilien-Dashboard",
        color="primary",
        dark=True,
        fluid=True
    )


def create_kpi_cards():
    cards = [
        dbc.Card(dbc.CardBody([html.H6("Anzahl Inserate"), html.H2(id="kpi-count")])),
        dbc.Card(dbc.CardBody([html.H6("Durchschnittspreis"), html.H2(id="kpi-avgprice")])),
        dbc.Card(dbc.CardBody([html.H6("Durchschn. Wohnfläche"), html.H2(id="kpi-avgarea")]))
    ]
    return dbc.Row([dbc.Col(card, md=4) for card in cards], className="mb-4")

filter_card = dbc.Card([
    dbc.CardHeader(html.H5("Filter")),
    dbc.CardBody([
        dbc.Label("Gemeinde", html_for='municipality-dropdown'),
        dcc.Dropdown(id='municipality-dropdown', multi=True, placeholder='Auswählen…'),
        html.Br(),
        dbc.Label("Immobilientyp", html_for='type-dropdown'),
        dcc.Dropdown(id='type-dropdown', multi=True, placeholder='Auswählen…'),
        html.Br(),
        dbc.Label("Preisspanne (CHF)", html_for='price-slider'),
        dcc.RangeSlider(
            id='price-slider',
            step=step,
            tooltip={'placement': 'bottom'},
            allowCross=False
        )
    ])
], className="mb-4")

def create_footer():
    return dbc.Container(
        html.Footer("2025 Einblicke in den Schweizer Immobilienmarkt",
                    className="text-center text-muted py-3"),
        fluid=True
    )

# App und Layout
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

app.layout = dbc.Container([
    dcc.Store(id='history-store', data={
    'past': [],
    'present': {'munis': [], 'types': [], 'price': [slider_min, slider_max]},
    'future': []}),
    create_navbar(),
    dbc.Row([
        dbc.Col(dbc.Button('↶ Rückgängig', id='undo-btn', color='secondary', disabled=True), width='auto'),
        dbc.Col(dbc.Button('↷ Wiederherstellen', id='redo-btn', color='secondary', disabled=True), width='auto')
    ], className='mb-2'),
    create_kpi_cards(),
    dbc.Row([
        dbc.Col(filter_card, md=4),
        dbc.Col(dcc.Loading(dcc.Graph(id='map-overview', style={'height': '60vh'})), md=8)
    ], className="mb-4"),
    dbc.Row([
        dbc.Col(dcc.Loading(dcc.Graph(id='histogram-price')), md=6),
        dbc.Col(dcc.Loading(dcc.Graph(id='scatter-overview', config={'modeBarButtonsToAdd': ['lasso2d']})), md=6)
    ], className="mb-4"),
    dbc.Collapse(
        dash_table.DataTable(
            id='details-table',
            columns=[
                {'name': 'Gemeinde', 'id': 'Municipality'},
                {'name': 'Typ', 'id': 'type_unified'},
                {'name': 'Fläche (m²)', 'id': 'Living_area_unified'},
                {'name': 'Preis (CHF)', 'id': 'price'},
                {'name': 'Beschreibung', 'id': 'description'},
                {'name': 'Link', 'id': 'url', 'presentation': 'markdown'}
            ],
            page_size=5,
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
            style_cell={'padding': '8px', 'textAlign': 'left'},
            row_selectable='single'
        ),
        id='details-collapse', is_open=True
    ),
    create_footer()
], fluid=True, className="p-4")

# Hilfsfunktion
def filter_df(df, municipalities, types, price_range):
    dff = df[df['price'].between(price_range[0], price_range[1])]
    if municipalities:
        dff = dff[dff['Municipality'].isin(municipalities)]
    if types:
        dff = dff[dff['type_unified'].isin(types)]
    return dff

# --- Neuer, kombinierter Callback für Filter-Optionen und Slider-Wert ---
@app.callback(
    Output('municipality-dropdown', 'options'),
    Output('type-dropdown', 'options'),
    Output('price-slider', 'min'),
    Output('price-slider', 'max'),
    Output('price-slider', 'value'),
    Input('history-store', 'data'),
    Input('histogram-price', 'clickData'),
    State('price-slider', 'value'),
    prevent_initial_call=False
)
def update_filters_and_slider(history, clickData, current_price):
    ctx = callback_context
    trigger = ctx.triggered[0]['prop_id'].split('.')[0]

    # Basis-Filter-State (z.B. aus Undo/Redo)
    state = history.get('present') or {
        'munis': [], 'types': [], 'price': [slider_min, slider_max]
    }

    # Wenn Histogramm geklickt: neuen Preisbereich setzen
    if trigger == 'histogram-price' and clickData:
        start = int(clickData['points'][0]['x'])
        price_val = [start, min(start + step, slider_max)]
    else:
        price_val = state['price']

    # DataFrame filtern
    dff = filter_df(df, state['munis'], state['types'], price_val)

    # Neue Dropdown-Optionen
    muni_opts = [{'label': m, 'value': m} for m in sorted(dff['Municipality'].dropna().unique())]
    type_opts = [{'label': t, 'value': t} for t in sorted(dff['type_unified'].dropna().unique())]

    return (
        muni_opts,
        type_opts,
        slider_min,
        slider_max,
        price_val
    )

# 2) Kombinierter Undo/Redo-Callback bleibt unverändert
@app.callback(
    Output('municipality-dropdown', 'value'),
    Output('type-dropdown', 'value'),
    Output('undo-btn', 'disabled'),
    Output('redo-btn', 'disabled'),
    Output('history-store', 'data'),
    Input('undo-btn', 'n_clicks'),
    Input('redo-btn', 'n_clicks'),
    State('history-store', 'data'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def on_undo_redo(n_undo, n_redo, history):
    ctx = callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    btn = ctx.triggered[0]['prop_id'].split('.')[0]
    if btn == 'undo-btn':
        if not history['past']:
            raise dash.exceptions.PreventUpdate
        prev = history['past'][-1]
        new_past = history['past'][:-1]
        new_future = [history['present']] + history['future']
    else:  # redo-btn
        if not history['future']:
            raise dash.exceptions.PreventUpdate
        prev = history['future'][0]
        new_future = history['future'][1:]
        new_past = history['past'] + [history['present']]

    new_history = {'past': new_past, 'present': prev, 'future': new_future}
    disabled_undo = not new_history['past']
    disabled_redo = not new_history['future']

    return prev['munis'], prev['types'], disabled_undo, disabled_redo, new_history

# 3) KPI-Callback
@app.callback(
    Output('kpi-count', 'children'),
    Output('kpi-avgprice', 'children'),
    Output('kpi-avgarea', 'children'),
    Input('municipality-dropdown', 'value'),
    Input('type-dropdown', 'value'),
    Input('price-slider', 'value'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def update_kpis(munis, types, price_rng):
    dff = filter_df(df, munis, types, price_rng)
    count = len(dff)
    avg_price = f"CHF {dff['price'].mean():,.0f}" if count else "-"
    avg_area = f"{dff['Living_area_unified'].mean():.0f} m²" if count else "-"
    return count, avg_price, avg_area

# 4) Graph-Update mit Animation
@app.callback(
    Output('map-overview', 'figure'),
    Output('histogram-price', 'figure'),
    Output('scatter-overview', 'figure'),
    Input('municipality-dropdown', 'value'),
    Input('type-dropdown', 'value'),
    Input('price-slider', 'value'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def update_graph(munis, types, price_rng):
    dff = filter_df(df, munis, types, price_rng)
    dff_plot = dff.rename(columns={
        'Municipality':'Gemeinde',
        'type_unified':'Immobilientyp',
        'price':'Preis (CHF)',
        'Living_area_unified':'Wohnfläche (m²)',
        'description':'Beschreibung'
    })
    fig_map = px.scatter_mapbox(
        dff_plot, lat='lat', lon='lon',
        color='Preis (CHF)', size='Wohnfläche (m²)', size_max=15,
        hover_name='Gemeinde',
        hover_data={'Immobilientyp':True,'Preis (CHF)':True,'lat':False,'lon':False},
        mapbox_style='carto-positron', zoom=6,
        labels={'Preis (CHF)':'Preis','Wohnfläche (m²)':'Fläche'}
    )
    fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, transition={'duration':500})

    fig_hist = px.histogram(dff_plot, x='Preis (CHF)', title='Preisverteilung (CHF)')
    fig_hist.update_layout(
        margin={'r':0,'t':30,'l':0,'b':0},
        transition={'duration':500},
        clickmode='event+select'
    )

    fig_scatter = px.scatter(
        dff_plot, x='Wohnfläche (m²)', y='Preis (CHF)',
        color='Gemeinde',
        hover_name='Gemeinde',
        hover_data={'Wohnfläche (m²)':True,'Preis (CHF)':True},
        labels={'Wohnfläche (m²)':'Fläche','Preis (CHF)':'Preis'}
    )
    fig_scatter.update_traces(marker=dict(opacity=0.7, line=dict(width=0.5)))
    fig_scatter.update_layout(
        margin={'r':0,'t':0,'l':0,'b':0},
        xaxis_title='Wohnfläche (m²)',
        yaxis_title='Preis (CHF)',
        transition={'duration':500}
    )

    return fig_map, fig_hist, fig_scatter

# 5) Punkte → Details-Tabelle
@app.callback(
    Output('details-table', 'data'),
    Output('details-collapse', 'is_open'),
    Input('scatter-overview', 'clickData'),
    Input('scatter-overview', 'selectedData'),
    State('details-collapse', 'is_open'),
    prevent_initial_call=True,
    allow_duplicate=True
)
def display_details(clickData, selectedData, is_open):
    if clickData and 'points' in clickData:
        x_val = clickData['points'][0]['x']
        y_val = clickData['points'][0]['y']
        sel = df[(df['Living_area_unified']==x_val)&(df['price']==y_val)]
        return sel.to_dict('records'), True
    if selectedData and 'points' in selectedData:
        xs = [p['x'] for p in selectedData['points']]
        ys = [p['y'] for p in selectedData['points']]
        sel = df[df['Living_area_unified'].isin(xs)&df['price'].isin(ys)]
        return sel.to_dict('records'), True
    return [], False

# 6) Affordance-Tooltips
for comp in ['municipality-dropdown', 'type-dropdown', 'price-slider']:
    app.layout.children.insert(
        3,
        dbc.Tooltip('Interaktives Element – hier Filtern!', target=comp)
    )


if __name__ == '__main__':
    app.run(debug=True, port=8053)
